### Notebook for concept detection in neural network

In [1]:
import numpy as np
import os
import sys
from tqdm import tqdm
import json
import random
import string

sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'src')))

import concepts
import env
from policy import ActorCriticNet

model_name = "net"
session_name = "starship"
board_name = "5x5"

agents_to_sample = [0, 150, 300, 450, 600, 750, 900, 1050]

full_model_path = f"../models/saved_sessions/board_size_5/{session_name}/"
board_size = 5

CONCEPT_FUNC = concepts.concept_eye

CONCEPT_NAME = concepts.concept_eye.__name__


2023-10-20 15:43:04.425996: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def load_model(full_name, model_name, epoch):
    model_path = full_name + model_name + "_" + str(epoch) + ".keras"
    model = ActorCriticNet(board_size, model_path)
    return model

agents = [load_model(full_model_path, model_name, epoch) for epoch in agents_to_sample]

In [39]:
def play_match(agents: list[ActorCriticNet], board_size, concept_function):
    go_env = env.GoEnv(board_size)
    state = go_env.reset()

    player_to_start = 1 if np.random.random() > 0.5 else 0
    current_player = player_to_start

    positive_cases = []
    negative_cases = []

    sample_ratio = 0.1

    game_over = False

    while not game_over:
        # Remove array index 3 and 5 from the current state making it an shape of (4, 5, 5)
        state_copy = np.delete(state, [3, 5], axis=0)
        
        if np.random.random() < sample_ratio:
            if concept_function(state):
                positive_cases.append(state_copy)
            else:
                negative_cases.append(state_copy)
        
        action = agents[current_player].best_action(state)

        state, _, game_over, _ = go_env.step(action)

        current_player = (current_player + 1) % 2
    
    return positive_cases, negative_cases

positive_cases = []
negative_cases = []

CASES_TO_SAMPLE = 250 # 25000

positive_bar = tqdm(total=CASES_TO_SAMPLE, desc="Positive cases")
while len(positive_cases) < CASES_TO_SAMPLE:
    pos, neg = play_match(agents, board_size, CONCEPT_FUNC)
    positive_cases.extend(pos)
    negative_cases.extend(neg)
    positive_bar.update(len(pos))

positive_cases = positive_cases[:CASES_TO_SAMPLE]
negative_cases = negative_cases[:CASES_TO_SAMPLE]

Positive cases: 2501it [14:08,  2.95it/s]0:00<?, ?it/s]
Positive cases: 100%|██████████| 250/250 [00:25<00:00, 11.05it/s]

In [40]:
positive_cases = np.array(positive_cases)
negative_cases = np.array(negative_cases)

print("Positive cases: ", positive_cases.shape)
print("Negative cases: ", negative_cases.shape)

Positive cases:  (250, 4, 5, 5)
Negative cases:  (209, 4, 5, 5)


In [41]:
# Find how many positive cases are identical to other positive cases and count the number of toatal cases
identical_cases = 0
for i in range(len(positive_cases)):
    for j in range(i + 1, len(positive_cases)):
        if np.array_equal(positive_cases[i], positive_cases[j]):
            identical_cases += 1

print(f"Total cases: {len(positive_cases)}")
print(f"Identical cases: {identical_cases}")

Total cases: 250
Identical cases: 947


In [42]:
epochs_to_look_at = [0, 150, 300, 450, 600, 750, 900, 1050]

POSITIONS_TO_CONSIDER = 400 #40000
VALIDATION_POSITIONS = 10000 #10000

for epoch in epochs_to_look_at:
    path = full_model_path + model_name + "_" + str(epoch) + ".keras"
    model = ActorCriticNet(board_size, path)

    # Will have a length equal to the sum of the numer of rows in the positive and negative cases arrays
    # And will contain 1s for positive cases and 0s for negative cases
    # Is used as labels/targets for the regression
    all_cases = np.concatenate([positive_cases, negative_cases])
    all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
    all_labels = np.array(all_labels)
    shuffled_indices = np.arange(all_labels.shape[0])

    np.random.shuffle(shuffled_indices)

    all_cases = all_cases[shuffled_indices]
    all_labels = all_labels[shuffled_indices]

    concept_presences = {}
    
    outputs = model.get_all_resblock_outputs(all_cases)

    # Merge outputs
    merged_outputs = []
    for output_batch in outputs:
        for i, output_layer in enumerate(output_batch):
            if len(merged_outputs) <= i:
                merged_outputs.append([])
            merged_outputs[i].extend(output_layer)

    for i, layer_output in enumerate(merged_outputs):
        merged_outputs[i] = np.array(merged_outputs[i])
    
    outputs = merged_outputs

    # Perform regression
    concept_presence_per_layer = []
    for (i, output) in enumerate(outputs):
        points = output.reshape((output.shape[0], np.prod(output.shape[1:])))
        # So one has (n, k) samples where n is the number of positions, and k is the total number of activation values in layer i.
        print(f"Performing regression for layer {i}")
        score = concepts.perform_regression(
            points=points[:POSITIONS_TO_CONSIDER], 
            targets=all_labels[:POSITIONS_TO_CONSIDER], 
            validation_points=points[POSITIONS_TO_CONSIDER:], 
            validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
            is_binary=True
        )
        concept_presence_per_layer.append(score)

        print(f"The presence of {CONCEPT_NAME} in resblock {i} is {score}")
    concept_presences[CONCEPT_NAME] = concept_presence_per_layer

    # Remove the files if they exist
    if os.path.exists("../concept_presences/{}/{}/{}/".format(board_name, session_name, CONCEPT_NAME)):
        # Test if epoch folder exists
        if os.path.exists("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch)):
            # Remove all files in the epoch folder
            for file in os.listdir("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch)):
                os.remove("../concept_presences/{}/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch, file))

    os.makedirs("../concept_presences", exist_ok=True)
    os.makedirs("../concept_presences/{}".format(board_name), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}".format(board_name, session_name), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch), exist_ok=True)

    random_suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

    # Save concept presences in json file
    with open("../concept_presences/{}/{}/{}/{}/{}.json".format(board_name, session_name, CONCEPT_NAME, epoch, random_suffix), "w") as f:
        json.dump(concept_presences[CONCEPT_NAME], f)




Getting resblock outputs: 100%|██████████| 15/15 [00:00<00:00, 196.98it/s]

Performing regression for layer 0
Epoch 1/50


13/13 [==============================] - 1s 13ms/step - loss: 1.0359 - val_loss: 1.0352
Epoch 2/50
13/13 [==============================] - 0s 3ms/step - loss: 1.0216 - val_loss: 1.0199
Epoch 3/50
13/13 [==============================] - 0s 3ms/step - loss: 1.0076 - val_loss: 1.0058
Epoch 4/50
13/13 [==============================] - 0s 3ms/step - loss: 0.9945 - val_loss: 0.9918
Epoch 5/50
13/13 [==============================] - 0s 3ms/step - loss: 0.9818 - val_loss: 0.9788
Epoch 6/50
13/13 [==============================] - 0s 3ms/step - loss: 0.9696 - val_loss: 0.9670
Epoch 7/50
13/13 [==============================] - 0s 3ms/step - loss: 0.9581 - val_loss: 0.9552
Epoch 8/50
13/13 [==============================] - 0s 3ms/step - loss: 0.9470 - val_loss: 0.9436
Epoch 9/50
13/13 [==============================] - 0s 3ms/step - loss: 0.9358 - val_loss: 0.9332
Epoch 10/50
13/13 [==============================] - 0s 3ms/step - loss: 0.9255 - val_loss: 0.9227
Epoch 11/50
13/13 [==========

Positive cases: 100%|██████████| 250/250 [00:45<00:00, 11.05it/s]

13/13 [==============================] - 0s 10ms/step - loss: 0.8998 - val_loss: 0.8932
Epoch 2/50
13/13 [==============================] - 0s 3ms/step - loss: 0.8934 - val_loss: 0.8870
Epoch 3/50
13/13 [==============================] - 0s 3ms/step - loss: 0.8875 - val_loss: 0.8810
Epoch 4/50
13/13 [==============================] - 0s 3ms/step - loss: 0.8816 - val_loss: 0.8753
Epoch 5/50
13/13 [==============================] - 0s 5ms/step - loss: 0.8759 - val_loss: 0.8696
Epoch 6/50
13/13 [==============================] - 0s 4ms/step - loss: 0.8702 - val_loss: 0.8643
Epoch 7/50
13/13 [==============================] - 0s 3ms/step - loss: 0.8650 - val_loss: 0.8587
Epoch 8/50
13/13 [==============================] - 0s 3ms/step - loss: 0.8595 - val_loss: 0.8538
Epoch 9/50
13/13 [==============================] - 0s 3ms/step - loss: 0.8543 - val_loss: 0.8488
Epoch 10/50
13/13 [==============================] - 0s 3ms/step - loss: 0.8493 - val_loss: 0.8438
Epoch 11/50
13/13 [==========

...

Performing regression for layer 0
Epoch 1/50


13/13 [==============================] - 0s 10ms/step - loss: 0.9582 - val_loss: 0.9295
Epoch 2/50
13/13 [==============================] - 0s 3ms/step - loss: 0.9257 - val_loss: 0.9008
Epoch 3/50
13/13 [==============================] - 0s 3ms/step - loss: 0.8951 - val_loss: 0.8726
Epoch 4/50
13/13 [==============================] - 0s 3ms/step - loss: 0.8684 - val_loss: 0.8458
Epoch 5/50
13/13 [==============================] - 0s 3ms/step - loss: 0.8440 - val_loss: 0.8243
Epoch 6/50
13/13 [==============================] - 0s 3ms/step - loss: 0.8219 - val_loss: 0.8024
Epoch 7/50
13/13 [==============================] - 0s 3ms/step - loss: 0.8019 - val_loss: 0.7793
Epoch 8/50
13/13 [==============================] - 0s 3ms/step - loss: 0.7831 - val_loss: 0.7625
Epoch 9/50
13/13 [==============================] - 0s 3ms/step - loss: 0.7657 - val_loss: 0.7461
Epoch 10/50
13/13 [==============================] - 0s 3ms/step - loss: 0.7497 - val_loss: 0.7302
Epoch 11/50
13/13 [==========

Getting resblock outputs: 100%|██████████| 15/15 [00:00<00:00, 266.01it/s]

Performing regression for layer 0
Epoch 1/50


13/13 [==============================] - 0s 10ms/step - loss: 0.8741 - val_loss: 0.7763
Epoch 2/50
13/13 [==============================] - 0s 3ms/step - loss: 0.8368 - val_loss: 0.7559
Epoch 3/50
13/13 [==============================] - 0s 3ms/step - loss: 0.8036 - val_loss: 0.7382
Epoch 4/50
13/13 [==============================] - 0s 3ms/step - loss: 0.7752 - val_loss: 0.7222
Epoch 5/50
13/13 [==============================] - 0s 3ms/step - loss: 0.7507 - val_loss: 0.7073
Epoch 6/50
13/13 [==============================] - 0s 3ms/step - loss: 0.7292 - val_loss: 0.6937
Epoch 7/50
13/13 [==============================] - 0s 3ms/step - loss: 0.7100 - val_loss: 0.6811
Epoch 8/50
13/13 [==============================] - 0s 3ms/step - loss: 0.6928 - val_loss: 0.6683
Epoch 9/50
13/13 [==============================] - 0s 3ms/step - loss: 0.6774 - val_loss: 0.6572
Epoch 10/50
13/13 [==============================] - 0s 3ms/step - loss: 0.6627 - val_loss: 0.6459
Epoch 11/50
13/13 [==========